Hier werden die Portfolio-Methoden angewandt und mithilfe der Performance-Metriken bewertet. Die Ergebnisse werden in CSV-Dateien festgehalten. Das Akquirieren, modifizieren und beschreiben der Daten befindet sich der Übersicht halber im Notebook Datenbasis. Die Daten werden dort als CSV exportiert und hier als Dataframe wieder eingelesen.

Die Korrektheit der implementierten Methoden und Metriken wird in dem separaten Notebook Verifizierung anhand eines kleinen Teilbestandes aufgezeigt, siehe dazu auch beigefügte Excel-Dateien, in denen die Portfolio-Gewichte "per Hand" nachgerechnet und mit den Ergebnissen aus der Python-Implementierung abgeglichen werden.

ACHTUNG: LANGE LAUFZEIT!

## Importe

In [1]:
#Import der notwendigen Bibliotheken, Installation mit pip bereits erfolgt

import yfinance as yf
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import csv
from scipy.optimize import minimize
import math
import random
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import linkage, dendrogram
import riskfolio as rp
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import squareform
from riskfolio import HCPortfolio as hc
from pandas.tseries.offsets import MonthBegin
from pandas.tseries.offsets import MonthEnd

## Einlesen der Daten

In [2]:
#Bearbeitete JSE-Daten (Gesamtbestand)

df_JSE = pd.read_csv(
    'WhartonFinal_neu.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [3]:
df_JSE.info

<bound method DataFrame.info of                  8303  15604   15618  16179  16183  16291   16305  16355  \
Date                                                                       
2005-06-01  39.000000  62.11   74.50  14.50   8.60   4.42   82.40   9.95   
2005-06-02  38.800000  62.00   75.00  14.51   8.51   4.40   82.50   9.80   
2005-06-03  37.903333  60.79   75.64  14.50   8.50   4.37   82.61   9.90   
2005-06-06  37.006667  60.75   78.49  13.85   8.62   4.38   82.90   9.90   
2005-06-07  36.110000  60.90   77.90  13.89   8.67   4.43   83.45   9.90   
...               ...    ...     ...    ...    ...    ...     ...    ...   
2025-05-26        NaN    NaN  256.98    NaN  12.95    NaN  168.45    NaN   
2025-05-27        NaN    NaN  255.97    NaN  13.08    NaN  168.48    NaN   
2025-05-28        NaN    NaN  257.02    NaN  13.10    NaN  173.00    NaN   
2025-05-29        NaN    NaN  259.83    NaN  13.32    NaN  175.79    NaN   
2025-05-30        NaN    NaN  255.50    NaN  13.49    Na

## Definition der Methoden und Metriken

#### Equally-Weighted Portfolio

In [4]:
#Rückgabe gleicher Gewichte anhand der Anzahl der Unternehmen (Input Returns)

def equally_weighted(returns):
    anzahl_assets = returns.shape[1]

    w0 = 1/anzahl_assets
    gewichte = np.full(anzahl_assets, w0)

    return gewichte

#### Minimum-Variance Portfolio

In [5]:
#Minimum-Variance Portfolio
#Code adaptiert von https://medium.com/@BorisGerat/mean-variance-and-minimum-variance-portfolio-models-in-python-64a5c6b57b2d

def min_variance_opt(matrix):
    
    #Compute the minimum-variance portfolio weights subject to the constraints of:
    #- No short-selling (weights must be >= 0)
    #- Full investment (sum of weights must equal 1)
    
    number_of_assets = matrix.shape[0]

    # Define the objective function to minimize portfolio variance
    def objective(w):
        portfolio_variance = np.dot(w, matrix @ w)
        return portfolio_variance

    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(number_of_assets)]

    # Initial guess: Equal weights for all assets
    w0 = np.ones(number_of_assets) / number_of_assets

    # Solve the optimization problem using Sequential Least Squares Quadratic Programming (SLSQP)
    res = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return res.x  # Return the optimal weights

#### Maximum Diversification Portfolio

In [6]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def calc_max_diversification_ratio(weights, matrix):
    
    # gewichtete Volatilität
    average_vol = np.dot(np.sqrt(np.diag(matrix)), weights.T)
    
    # portfolio volatilität
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_vol = np.sqrt(portfolio_variance)
    
    #Bestimmung Ratio
    diversification_ratio = average_vol/portfolio_vol
    
    # Negativer Wert für Minimierung (Maximizieren = Minimieren -)
    return -diversification_ratio

In [7]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def max_diversification_portfolio(matrix):
    
    anzahl_assets = matrix.shape[0]
    
    w0 = 1/anzahl_assets
    w = np.full(anzahl_assets, w0)
    
    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    
    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(anzahl_assets)]
    
    res = minimize(calc_max_diversification_ratio, w, bounds=bounds, args=(matrix), method='SLSQP', constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    return res.x

#### Inverse Volatility Portfolio

In [8]:
#Code adaptiert von https://www.kaggle.com/code/vijipai/lesson-4-traditional-portfolio-construction-method

def inverse_volatility_portfolio (matrix):
    
    standardabweichungen = np.sqrt(np.diagonal(matrix))
    invertierte_std = 1/standardabweichungen
    summe_invertierte_std = np.sum(invertierte_std)
    
    inverse_volatility_gewichte = invertierte_std / summe_invertierte_std
    
    return inverse_volatility_gewichte

#### Equal Risk Contribution Portfolio

In [9]:
#Code adaptiert von https://medium.com/@deepml1818/building-a-python-based-risk-parity-portfolio-for-trading-40441ecdd84d

def calculate_portfolio_volatility(weights, matrix):
    
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_volatility = np.sqrt(portfolio_variance)
    return portfolio_volatility

def calculate_risk_contribution(weights, matrix):
    portfolio_volatility = calculate_portfolio_volatility(weights, matrix)
    marginal_contribution_zaehler = np.dot(matrix, weights)
    risk_contribution = np.multiply(weights, marginal_contribution_zaehler) / portfolio_volatility
    return risk_contribution

def risk_parity_objective(weights, matrix):
    risk_contribution = calculate_risk_contribution(weights, matrix)
    target_risk = np.mean(risk_contribution)
    return np.sum((risk_contribution - target_risk) ** 2)

In [10]:
def get_risk_parity_weights(matrix):
    
    number_of_assets = matrix.shape[0]
    w0 = 1/number_of_assets
    initial_weights = np.full(number_of_assets, w0)
    
    constraints = ({'type': 'eq', 'fun': lambda initial_weights: np.sum(initial_weights) - 1})
    bounds = [[0, 1] for _ in range(number_of_assets)]
    
    result = minimize(risk_parity_objective, initial_weights, args=(matrix), method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return result.x

#### Hierarchical Risk Parity

In [11]:
#Aus dem Paper von Lopez De Prado (2016)
def getIVP(cov,**kargs): # Compute the inverse-variance portfolio 
    ivp=1./np.diag(cov) 
    ivp/=ivp.sum() 
    return ivp

In [12]:
#------------------------------------------------------------------------------ 
def getClusterVar(cov,cItems): # Compute variance per cluster 
    cov_=cov.loc[cItems,cItems] # matrix slice 
    w_=getIVP(cov_).reshape(-1,1) 
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0] 
    return cVar 

In [13]:
#------------------------------------------------------------------------------ 
def getQuasiDiag(link): # Sort clustered items by distance 
    link=link.astype(int) 
    sortIx=pd.Series([link[-1,0],link[-1,1]]) 
    numItems=link[-1,3] # number of original items 
    while sortIx.max()>=numItems: 
        sortIx.index=range(0,sortIx.shape[0]*2,2) # make space 
        df0=sortIx[sortIx>=numItems] # find clusters
        i=df0.index;j=df0.values-numItems 
        sortIx[i]=link[j,0] # item 1 
        df0=pd.Series(link[j,1],index=i+1) 
        sortIx = pd.concat([sortIx, df0]) # item 2  ####Anpassung wegen neuer Pandas-Version!
        sortIx=sortIx.sort_index() # re-sort 
        sortIx.index=range(sortIx.shape[0]) # re-index 
    return sortIx.tolist() 

In [14]:
#------------------------------------------------------------------------------ 
def getRecBipart(cov,sortIx): # Compute HRP allocation
    w = pd.Series(1,index=sortIx) 
    cItems=[sortIx] # initialize all items in one cluster 
    while len(cItems)>0: 
        #Bi-Section der Cluster
        cItems = [i[j:k] 
                  for i in cItems 
                  for j, k in ((0, len(i)//2), (len(i)//2, len(i))) 
                  if len(i) > 1]
        for i in range(0, len(cItems), 2): # parse in pairs 
            cItems0=cItems[i] # cluster 1 
            cItems1=cItems[i+1] # cluster 2 
            cVar0=getClusterVar(cov,cItems0) 
            cVar1=getClusterVar(cov,cItems1) 
            alpha=1-cVar0/(cVar0+cVar1) 
            w[cItems0]*=alpha # weight 1 
            w[cItems1]*=1-alpha # weight 2 
    return w 

In [15]:
#------------------------------------------------------------------------------ 
def correlDist(corr): 
    # A distance matrix based on correlation, where 0<=d[i,j]<=1 
    # This is a proper distance metric 
    dist=((1-corr)/2.)**.5 # distance matrix 
    
    #Adjustierung zur condensed_distance_matrix
    condensed_distance_matrix = squareform(X=dist, checks=False)
    
    return condensed_distance_matrix

In [16]:
def hrp_main(corr, cov, returns): 
    
    #3) cluster 
    dist=correlDist(corr) 
    link=sch.linkage(dist,'single') 
    sortIx=getQuasiDiag(link) 
    sortIx=corr.index[sortIx].tolist() # recover labels 
    df0=corr.loc[sortIx,sortIx] # reorder 
    
    #4) Capital allocation 
    hrp=getRecBipart(cov,sortIx) 
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    hrp_sortiert = hrp.reindex(original_reihenfolge)
    
    array = hrp_sortiert.T.values

    return array

#### Hierarchical Equal Risk Contribution Portfolio

In [17]:
#Es wird auf die Implementierung der Riskfolio-Lib zurückgegriffen
#Code adaptiert von #Code adaptiert von https://medium.com/@orenji.eirl/hierarchical-equal-risk-contribution-with-python-and-riskfolio-lib-ec45dd0f9899

def herc (returns):

    # Building the portfolio object
    port = hc(returns=returns)

    # Estimate optimal portfolio:

    model='HERC'
    correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          correlation=correlation,
                          rm=rm,
                          rf=rf,
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

#### Vergleichs-Metriken

In [18]:
def gesamt_return(gewichte, returns):

    daily_returns = returns @ gewichte
    total = np.prod(1 + daily_returns) - 1
    
    return total

In [19]:
def portfolio_return (gewichte, returns):
    
    avg_returns = []
    avg_returns = returns.mean()
    
    pf_return = sum([gewicht * rendite for gewicht, rendite in zip(gewichte, avg_returns)])
    
    return pf_return

In [20]:
def portfolio_variance (gewichte, returns):
    daily_returns = []
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    pf_var = np.var(daily_returns)
    
    return pf_var

In [21]:
def portfolio_volatility (gewichte, returns):
    variance = portfolio_variance(gewichte, returns)
    volatility = np.sqrt(variance)
    
    return volatility

In [22]:
def annualized_return(gewichte, returns):
    daily_returns = returns @ gewichte
    annual = (np.prod(1 + daily_returns))**(252/daily_returns.shape[0]) - 1
    
    return annual

In [23]:
def annualized_portfolio_return(gewichte, returns):
    pf_return = portfolio_return(gewichte, returns)
    
    return pf_return*252

In [24]:
def annualized_portfolio_volatility(gewichte, returns):
    pf_volatility = portfolio_volatility(gewichte, returns)
    
    return pf_volatility*np.sqrt(252)

In [25]:
def sharpe_ratio (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [26]:
def sharpe_ratio_one (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.01
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [27]:
def sharpe_ratio_two (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.02
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [28]:
def annualized_sharpe_ratio (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [29]:
def annualized_sharpe_ratio_one (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.01
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [30]:
def annualized_sharpe_ratio_two (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.02
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [31]:
def sortino_ratio (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [32]:
def sortino_ratio_one (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.01
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [33]:
def sortino_ratio_two (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.02
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [34]:
def annualized_sortino_ratio (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [35]:
def annualized_sortino_ratio_one (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.01
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [36]:
def annualized_sortino_ratio_two (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.02
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [37]:
def maximum_drawdown (gewichte, returns):
    daily_returns = returns @ gewichte
    returns_kumuliert = (1 + daily_returns).cumprod()
    hochpunkte = returns_kumuliert.cummax()
    
    drawdowns = (returns_kumuliert - hochpunkte) / hochpunkte
    max_drawdown = drawdowns.min()
    
    #print(drawdowns)
    return max_drawdown

In [38]:
def calmar_ratio (gewichte, returns):
    pf_return = portfolio_return(gewichte, returns)
    max_drawdown = maximum_drawdown(gewichte, returns)
    
    calmar_ratio = pf_return / (-max_drawdown)
    
    return calmar_ratio

In [39]:
def value_at_risk_func(gewichte, returns):
    daily_returns = returns @ gewichte
    sorted_daily_returns = daily_returns.sort_values(ascending = True)
    value_at_risk = np.percentile(sorted_daily_returns, 100*0.05)
    return value_at_risk

In [40]:
#https://github.com/malctaylor15/VaR-and-Expected-Shortfall-Python/blob/master/Expected%20Shortfall%20and%20Value%20at%20Risk.ipynb

def expected_shortfall(gewichte, returns):
    daily_returns = returns @ gewichte
    sorted_daily_returns = daily_returns.sort_values(ascending = True)
    value_at_risk = value_at_risk_func(gewichte, returns)
    
    expected_shortfall = daily_returns[daily_returns <= value_at_risk].mean()
    
    return expected_shortfall

#### Sammel-Funktionen zum Berechnen und Speichern der Metriken

In [41]:
def metriken_equally_weighted(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile1 = {"Methode": "Equally-Weighted",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile1[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile1])], ignore_index=True)

In [42]:
def metriken_minimum_variance(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile2 = {"Methode": "Minimum_Variance",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile2[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile2])], ignore_index=True)

In [43]:
def metriken_max_diversification(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile3 = {"Methode": "Max-Diversification",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile3[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile3])], ignore_index=True)

In [44]:
def metriken_inverse_volatility(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile4 = {"Methode": "Inverse-Volatility",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile4[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile4])], ignore_index=True)

In [45]:
def metriken_risk_parity(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile5 = {"Methode": "Risk-Parity",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile5[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile5])], ignore_index=True)

In [46]:
def metriken_hrp(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile6 = {"Methode": "HRP",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile6[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile6])], ignore_index=True)

In [47]:
def metriken_herc(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile7 = {"Methode": "HERC",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile7[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile7])], ignore_index=True)

## Erzeugung Ergebnisse S&P500

## Ergebnis-Erzeugung JSE

In [48]:
returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                 8303     15604     15618     16179     16183     16291  \
Date                                                                     
2005-06-02 -0.005128 -0.001771  0.006711  0.000690 -0.010465 -0.004525   
2005-06-03 -0.023110 -0.019516  0.008533 -0.000689 -0.001175 -0.006818   
2005-06-06 -0.023657 -0.000658  0.037678 -0.044828  0.014118  0.002288   
2005-06-07 -0.024230  0.002469 -0.007517  0.002888  0.005800  0.011416   
2005-06-08 -0.027970 -0.003284 -0.005135  0.006479  0.014994  0.006772   
...              ...       ...       ...       ...       ...       ...   
2025-05-26       NaN       NaN  0.013928       NaN -0.001542       NaN   
2025-05-27       NaN       NaN -0.003930       NaN  0.010039       NaN   
2025-05-28       NaN       NaN  0.004102       NaN  0.001529       NaN   
2025-05-29       NaN       NaN  0.010933       NaN  0.016794       NaN   
2025-05-30       NaN       NaN -0.016665       NaN  0.012763       NaN   

     

In [49]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 24
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    #Inf-Werte in NaN umwandeln
    training_daten = training_daten.replace([np.inf, -np.inf], np.nan)
    #Spalten entfernen, die nachher noch NaN enthalten
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 126
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    JSE_gewichte_equally_weighted = equally_weighted(training_daten)
    JSE_gewichte_min_variance= min_variance_opt(cov_matrix)
    JSE_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    JSE_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    JSE_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    JSE_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    JSE_gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": JSE_gewichte_equally_weighted,
    "Minimum-Variance": JSE_gewichte_min_variance,
    "Maximum-Diversification": JSE_gewichte_max_diversification,
    "Inverse-Volatility": JSE_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": JSE_gewichte_risk_parity,
    "HRP": JSE_gewichte_hrp,
    "HERC": JSE_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
# Alle Testperioden zusammenführen
portfolio_returns_df_JSE = pd.concat(portfolio_returns_list)
portfolio_returns_df_JSE = portfolio_returns_df_JSE.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.820511576969558e-05
            Iterations: 225
            Function evaluations: 20925
            Gradient evaluations: 225
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.7571745985998604
            Iterations: 60
            Function evaluations: 5642
            Gradient evaluations: 60
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.239010666692647e-09
            Iterations: 54
            Function evaluations: 5024
            Gradient evaluations: 54
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9092587886730358e-05
            Iterations: 244
            Function evaluations: 22448
            Gradient evaluations: 244
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.74

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.730343499235631
            Iterations: 65
            Function evaluations: 6633
            Gradient evaluations: 65
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2478416907656169e-08
            Iterations: 55
            Function evaluations: 5556
            Gradient evaluations: 55
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8341277543752784e-05
            Iterations: 190
            Function evaluations: 19190
            Gradient evaluations: 190
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.945538688837881
            Iterations: 73
            Function evaluations: 7453
            Gradient evaluations: 73
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4359214264927051e-08
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0287676711058083e-08
            Iterations: 121
            Function evaluations: 12221
            Gradient evaluations: 121
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.9946000159954804e-05
            Iterations: 196
            Function evaluations: 19601
            Gradient evaluations: 196
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.9877467505227346
            Iterations: 68
            Function evaluations: 6867
            Gradient evaluations: 68
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.052537546039716e-08
            Iterations: 120
            Function evaluations: 12000
            Gradient evaluations: 120
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.521433046378554e-05
            Iterations: 222
            Function evaluations: 20647
            Gradient evaluations: 222
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.960945330432484
            Iterations: 57
            Function evaluations: 5363
            Gradient evaluations: 57
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.163592037955379e-08
            Iterations: 142
            Function evaluations: 13206
            Gradient evaluations: 142
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.6818584772663408e-05
            Iterations: 222
            Function evaluations: 20424
            Gradient evaluations: 222
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.

Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.032293635222356
            Iterations: 60
            Function evaluations: 5585
            Gradient evaluations: 60
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1958001024855976e-08
            Iterations: 45
            Function evaluations: 4140
            Gradient evaluations: 45
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.079945755293671e-05
            Iterations: 207
            Function evaluations: 19044
            Gradient evaluations: 207
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.118621367860547
            Iterations: 60
            Function evaluations: 5583
            Gradient evaluations: 60
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1279495712571055e-08
         

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.729901975002583
            Iterations: 108
            Function evaluations: 13493
            Gradient evaluations: 104
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.3582088545399258e-08
            Iterations: 69
            Function evaluations: 8764
            Gradient evaluations: 69
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2468663142020655e-05
            Iterations: 302
            Function evaluations: 37448
            Gradient evaluations: 302
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.661969317305366
            Iterations: 83
            Function evaluations: 9912
            Gradient evaluations: 79
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5037052215473993e-08
     

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.4608527385469147e-08
            Iterations: 86
            Function evaluations: 11009
            Gradient evaluations: 86
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.67501043539987e-05
            Iterations: 289
            Function evaluations: 36992
            Gradient evaluations: 289
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -4.697915857645089
            Iterations: 92
            Function evaluations: 11385
            Gradient evaluations: 88
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.471198403128674e-08
            Iterations: 75
            Function evaluations: 9601
            Gradient evaluations: 75
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.417813930258833e-06
            Iterations: 430
            Function evaluations: 55041
            Gradient evaluations: 430
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.378712689116157
            Iterations: 127
            Function evaluations: 16153
            Gradient evaluations: 123
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.630793805114307e-08
            Iterations: 38
            Function evaluations: 4865
            Gradient evaluations: 38
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.195230448131827e-06
            Iterations: 447
            Function evaluations: 56769
            Gradient evaluations: 447
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.4

Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.25976230885995
            Iterations: 88
            Function evaluations: 11127
            Gradient evaluations: 84
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.768963095580816e-08
            Iterations: 45
            Function evaluations: 5895
            Gradient evaluations: 45
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.474095560585033e-06
            Iterations: 359
            Function evaluations: 45952
            Gradient evaluations: 359
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.052828729081215
            Iterations: 92
            Function evaluations: 11442
            Gradient evaluations: 88
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8712060168077682e-08
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0927111607943177e-08
            Iterations: 43
            Function evaluations: 5762
            Gradient evaluations: 43
2022-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.395996582317136e-06
            Iterations: 390
            Function evaluations: 52261
            Gradient evaluations: 390
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.229254532080386
            Iterations: 86
            Function evaluations: 11512
            Gradient evaluations: 85
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.054402234275208e-08
            Iterations: 42
            Function evaluations: 5628
            Gradient evaluations: 42
2023-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.843192

In [50]:
#Exportieren nach CSV
portfolio_returns_df_JSE.to_csv("returnsWharton_neu_2jahre_f.csv")

## Ledoit-Wolf Shrinkage

In [51]:
#Definition von Ledoit-Shrinkage

def ledoit_shrink (returns):
    
    shrinkage = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix = pd.DataFrame(
        shrinkage.covariance_,
        index = returns.columns,
        columns = returns.columns)
    
    #print(shrinkage.shrinkage_)
    
    return shrinkage_matrix

### Auswertung S&P500 mit Ledoit Wolf

### Auswertung JSE mit Ledoit Wolf

In [52]:
returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                 8303     15604     15618     16179     16183     16291  \
Date                                                                     
2005-06-02 -0.005128 -0.001771  0.006711  0.000690 -0.010465 -0.004525   
2005-06-03 -0.023110 -0.019516  0.008533 -0.000689 -0.001175 -0.006818   
2005-06-06 -0.023657 -0.000658  0.037678 -0.044828  0.014118  0.002288   
2005-06-07 -0.024230  0.002469 -0.007517  0.002888  0.005800  0.011416   
2005-06-08 -0.027970 -0.003284 -0.005135  0.006479  0.014994  0.006772   
...              ...       ...       ...       ...       ...       ...   
2025-05-26       NaN       NaN  0.013928       NaN -0.001542       NaN   
2025-05-27       NaN       NaN -0.003930       NaN  0.010039       NaN   
2025-05-28       NaN       NaN  0.004102       NaN  0.001529       NaN   
2025-05-29       NaN       NaN  0.010933       NaN  0.016794       NaN   
2025-05-30       NaN       NaN -0.016665       NaN  0.012763       NaN   

     

In [53]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 24
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
     #Inf-Werte in NaN umwandeln
    training_daten = training_daten.replace([np.inf, -np.inf], np.nan)
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 126
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    shrinked_JSE_gewichte_equally_weighted = equally_weighted(training_daten)
    shrinked_JSE_gewichte_min_variance= min_variance_opt(cov_matrix)
    shrinked_JSE_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    shrinked_JSE_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    shrinked_JSE_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    shrinked_JSE_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    shrinked_JSE_gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": shrinked_JSE_gewichte_equally_weighted,
    "Minimum-Variance": shrinked_JSE_gewichte_min_variance,
    "Maximum-Diversification": shrinked_JSE_gewichte_max_diversification,
    "Inverse-Volatility": shrinked_JSE_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": shrinked_JSE_gewichte_risk_parity,
    "HRP": shrinked_JSE_gewichte_hrp,
    "HERC": shrinked_JSE_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_shrinked_JSE = pd.concat(portfolio_returns_list)
portfolio_returns_df_shrinked_JSE = portfolio_returns_df_shrinked_JSE.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.0262572115039524e-05
            Iterations: 168
            Function evaluations: 15624
            Gradient evaluations: 168
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.9083956900557366
            Iterations: 66
            Function evaluations: 6206
            Gradient evaluations: 66
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.338491895132295e-09
            Iterations: 42
            Function evaluations: 3907
            Gradient evaluations: 42
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.145187159505793e-05
            Iterations: 181
            Function evaluations: 16652
            Gradient evaluations: 181
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.88

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.8850178193239913
            Iterations: 70
            Function evaluations: 7144
            Gradient evaluations: 70
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.017464399662424e-08
            Iterations: 56
            Function evaluations: 5656
            Gradient evaluations: 56
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.0996215070808544e-05
            Iterations: 168
            Function evaluations: 16968
            Gradient evaluations: 168
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.1065965982469095
            Iterations: 77
            Function evaluations: 7871
            Gradient evaluations: 77
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1816767883298925e-08
       

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0818432022652659e-08
            Iterations: 99
            Function evaluations: 10000
            Gradient evaluations: 99
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.38981629247908e-05
            Iterations: 165
            Function evaluations: 16501
            Gradient evaluations: 165
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.057952049057705
            Iterations: 73
            Function evaluations: 7378
            Gradient evaluations: 73
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.004819596358127e-08
            Iterations: 110
            Function evaluations: 11000
            Gradient evaluations: 110
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.3233

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.8053096444969595e-05
            Iterations: 196
            Function evaluations: 18228
            Gradient evaluations: 196
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.9794430372057605
            Iterations: 59
            Function evaluations: 5551
            Gradient evaluations: 59
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2207714455061583e-08
            Iterations: 120
            Function evaluations: 11160
            Gradient evaluations: 120
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.9153957136539266e-05
            Iterations: 170
            Function evaluations: 15640
            Gradient evaluations: 170
Optimization terminated successfully    (Exit mode 0)
            Current function value: 

Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.082384821897027
            Iterations: 62
            Function evaluations: 5774
            Gradient evaluations: 62
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.085162003634942e-08
            Iterations: 45
            Function evaluations: 4140
            Gradient evaluations: 45
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1224917636279115e-05
            Iterations: 185
            Function evaluations: 17020
            Gradient evaluations: 185
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.1695995900307405
            Iterations: 64
            Function evaluations: 5959
            Gradient evaluations: 64
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0231938721862698e-08
        

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.66546769848881
            Iterations: 81
            Function evaluations: 10398
            Gradient evaluations: 81
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.0252462241101764e-08
            Iterations: 74
            Function evaluations: 9399
            Gradient evaluations: 74
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.9617102866844452e-05
            Iterations: 175
            Function evaluations: 21701
            Gradient evaluations: 175
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.592705430850045
            Iterations: 119
            Function evaluations: 14635
            Gradient evaluations: 115
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2017200677324128e-08
     

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1846232741667343e-08
            Iterations: 77
            Function evaluations: 9857
            Gradient evaluations: 77
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.6656734304864164e-05
            Iterations: 186
            Function evaluations: 23809
            Gradient evaluations: 186
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.666222287794148
            Iterations: 114
            Function evaluations: 14374
            Gradient evaluations: 110
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.0996252711856166e-08
            Iterations: 88
            Function evaluations: 11264
            Gradient evaluations: 88
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.3

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.167782465992958e-05
            Iterations: 266
            Function evaluations: 34048
            Gradient evaluations: 266
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.505359435738517
            Iterations: 139
            Function evaluations: 17826
            Gradient evaluations: 135
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.28004298494069e-08
            Iterations: 39
            Function evaluations: 4992
            Gradient evaluations: 39
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1153975627397469e-05
            Iterations: 272
            Function evaluations: 34545
            Gradient evaluations: 272
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.5

Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.315533743592712
            Iterations: 81
            Function evaluations: 10732
            Gradient evaluations: 81
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5685119715893274e-08
            Iterations: 45
            Function evaluations: 5895
            Gradient evaluations: 45
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1990030747127441e-05
            Iterations: 251
            Function evaluations: 32128
            Gradient evaluations: 251
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.130111516203622
            Iterations: 81
            Function evaluations: 10481
            Gradient evaluations: 81
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6746462305961375e-08
      

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1543243426703216e-08
            Iterations: 28
            Function evaluations: 3752
            Gradient evaluations: 28
2022-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0511990484496364e-05
            Iterations: 295
            Function evaluations: 39531
            Gradient evaluations: 295
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.348243297738249
            Iterations: 94
            Function evaluations: 12600
            Gradient evaluations: 93
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1104598645153818e-08
            Iterations: 27
            Function evaluations: 3618
            Gradient evaluations: 27
2023-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.7188

In [54]:
#Exportieren nach CSV
portfolio_returns_df_shrinked_JSE.to_csv("returnsWhartonshrinked_neu_2jahre_f.csv")

### Reduzierter S&P500-Bestand

### Reduziert und Ledoit-Shrinkage